In [1]:
import sqlite3
import requests
import time

import os

In [3]:
WB_TOKEN = os.getenv('WB_TOKEN')
DB_NAME = "zip.db"

In [4]:
# Параметры подключения к базе

WB_API_URL = "https://discounts-prices-api.wildberries.ru/api/v2/upload/task"
HEADERS = {"Authorization": f"Bearer {WB_TOKEN}", "Content-Type": "application/json"}

# Подключение к базе и выборка записей
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()

query = """
    SELECT nm_id, new_price, new_discount
    FROM products
    WHERE available_count = 5
"""
cursor.execute(query)
rows = cursor.fetchall()
conn.close()

# Формируем список товаров для обновления цен и скидок
goods_data = [{"nmID": nm_id, "price": new_price, "discount": new_discount} for nm_id, new_price, new_discount in rows]

# Отправка данных батчами по 1000 товаров (API поддерживает максимум 1000 за запрос)
BATCH_SIZE = 1000
REQUEST_LIMIT = 10  # Максимум 10 запросов за 6 секунд
REQUEST_INTERVAL = 6 / REQUEST_LIMIT  # Интервал между запросами

for i in range(0, len(goods_data), BATCH_SIZE):
    batch = goods_data[i:i + BATCH_SIZE]

    payload = {"data": batch}
    response = requests.post(WB_API_URL, json=payload, headers=HEADERS)

    if response.status_code == 200:
        print(f"✅ Успешно обновлены цены и скидки для {len(batch)} товаров")
    else:
        print(f"❌ Ошибка при обновлении: {response.text}")

    time.sleep(REQUEST_INTERVAL)  # Задержка для соблюдения лимита API


✅ Успешно обновлены цены и скидки для 5 товаров


In [5]:

WB_API_N_URL = "https://marketplace-api.wildberries.ru/api/v3/stocks/{warehouseId}"
WAREHOUSE_ID = 1283008


In [6]:
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()

query = """
    SELECT skus, pcs, available_count
    FROM products
    WHERE skus IS NOT NULL
"""
cursor.execute(query)
rows = cursor.fetchall()
conn.close()

# Формируем список остатков
stocks_data = [
    {
        "sku": skus,
        "amount": (
            5 if available_count == 5 else
            0
        )
    }
    for skus, pcs, available_count in rows
]

In [7]:
stocks_data

[{'sku': '2042718081924', 'amount': 0},
 {'sku': '2042718081801', 'amount': 5},
 {'sku': '2042718081894', 'amount': 0},
 {'sku': '2042718081887', 'amount': 0},
 {'sku': '2042718081856', 'amount': 5},
 {'sku': '2042718081832', 'amount': 0},
 {'sku': '2042718081931', 'amount': 5},
 {'sku': '2042718081795', 'amount': 0},
 {'sku': '2042718081870', 'amount': 5},
 {'sku': '2042718081788', 'amount': 0},
 {'sku': '2042718081900', 'amount': 5}]

In [ ]:
BATCH_SIZE = 1000
REQUEST_LIMIT = 300
REQUEST_INTERVAL = 60 / REQUEST_LIMIT

for i in range(0, len(stocks_data), BATCH_SIZE):
    batch = stocks_data[i:i + BATCH_SIZE]

    payload = {"stocks": batch}
    url = WB_API_N_URL.format(warehouseId=WAREHOUSE_ID)
    response = requests.put(url, json=payload, headers=HEADERS)

    if response.status_code == 204:
        print(f"✅ Успешно обновлены остатки для {len(batch)} товаров")
    else:
        print(f"❌ Ошибка при обновлении: {response.text}")

    time.sleep(REQUEST_INTERVAL)

✅ Успешно обновлены остатки для 11 товаров
